In [1]:
from os import environ
import json

from dotenv import find_dotenv, load_dotenv
from elasticsearch import Elasticsearch
import spacy
from spellchecker import SpellChecker
from tqdm import tqdm


load_dotenv(find_dotenv())

True

In [176]:
es = Elasticsearch(
    environ.get('ELASTIC_URL'),
    basic_auth=(environ.get('ELASTIC_USER'), environ.get('ELASTIC_PASSWORD')),
    verify_certs=True
)

if es.ping():
    print("Подключение установлено.")
else:
    print("Не удалось подключиться к Elasticsearch.")

Подключение установлено.


In [177]:
index_name = "documents"

mappings = {
    "mappings": {
        "properties": {
            "document_name": {
                "type": "text"
            },
            "page_number": {
                "type": "integer"
            },
            "paragraph_number": {
                "type": "integer"
            },
            "text": {
                "type": "text"
            },
            "lemmatized_text": {
                "type": "text"
            }
        }
    }
}

if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=mappings)
    print(f"Индекс '{index_name}' создан")
else:
    print(f"Индекс '{index_name}' уже существует")

Индекс 'documents' создан


In [178]:
nlp = spacy.load("ru_core_news_sm")
spell = SpellChecker(language='ru')

In [179]:
def correct_text(text):
    words = text.split()
    corrected_words = []

    for word in words:
        corrected_word = spell.correction(word)
        if corrected_word is None:
            corrected_word = word
        corrected_words.append(corrected_word)
    
    return " ".join(corrected_words)

def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text

def preprocess_text(text): return lemmatize_text(correct_text(text))

In [ ]:
pdf_path = 'jsons/elasticsearch_data.json'

In [ ]:
with open(pdf_path, 'r', encoding='utf-8') as f:
    documents = json.load(f)

In [ ]:
page_number = 0
for doc in tqdm(documents[:150]): # всего 150 записей, потому что все записи даже одного документа обрабатывались бы час
    doc["lemmatized_text"] = preprocess_text(doc['text'])
    es.index(index=index_name, document=doc)

100%|██████████| 150/150 [04:33<00:00,  1.83s/it]


In [ ]:
query_text = "Железнодорожный транспотр" # ошибка допущена специально

In [183]:
query = {
    "query": {
        "match": {
            "lemmatized_text": {
                "query": preprocess_text(query_text),
                "minimum_should_match": "80%"
            } 
        }
    }
}

response = es.search(index=index_name, body=query)

for hit in response["hits"]["hits"]:
    print(f"Document Name: {hit['_source']['document_name']}")
    print(f"Page Number: {hit['_source']['page_number']}")
    print(f"Paragraph Number: {hit['_source']['paragraph_number']}")
    print(f"Text: {hit['_source']['text']}")
    print("------")


Document Name: 7enYF2uL5kFZlOOpQhLl0nUT91RjCbeR.pdf
Page Number: 11
Paragraph Number: 2
Text: "магистральный каркас городского пассажирского транспорта" -  совокупность видов транспорта общего пользования в рамках городской агломерации с выделенной дорожной или рельсовой инфраструктурой, в том числе метрополитен, трамвай, скоростной автобусный  и троллейбусный транспорт, а также связанная с ними сеть пригородного железнодорожного транспорта; 
------
Document Name: 7enYF2uL5kFZlOOpQhLl0nUT91RjCbeR.pdf
Page Number: 17
Paragraph Number: 4
Text: "транспортные средства" - воздушные суда, морские суда, суда  внутреннего плавания, смешанного (река-море) плавания,  железнодорожный подвижной состав, подвижной состав автомобильного и электрического городского пассажирского транспорта, а также перспективный подвижной состав (например, экранопланы и др.); 
------
Document Name: 7enYF2uL5kFZlOOpQhLl0nUT91RjCbeR.pdf
Page Number: 10
Paragraph Number: 3
Text: "единое транспортное пространство Российск

Сохраняю нижний блок для быстрого пересоздания индекса

In [ ]:
# response = es.indices.delete(index=index_name, ignore=[400, 404])
# print(response)